In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from tensorflow import keras
import time
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import categorical_crossentropy

train_x = pd.read_csv("../data/taxi_data/yellow_tripdata_2020-04.csv")
train_x

C:\Users\Sven\AppData\Local\Temp\ipykernel_17120\19442575.py:14: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  train_x = pd.read_csv("../data/taxi_data/yellow_tripdata_2020-04.csv")


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2020-04-01 00:41:22,2020-04-01 01:01:53,1.0,1.20,1.0,N,41,24,2,5.50,0.5,0.5,0.00,0.00,0.3,6.80,0.0,NaN
1,1,2020-04-01 00:56:00,2020-04-01 01:09:25,1.0,3.40,1.0,N,95,197,1,12.50,0.5,0.5,2.75,0.00,0.3,16.55,0.0,NaN
2,1,2020-04-01 00:00:26,2020-04-01 00:09:25,1.0,2.80,1.0,N,237,137,1,10.00,3.0,0.5,1.00,0.00,0.3,14.80,2.5,NaN
3,1,2020-04-01 00:24:38,2020-04-01 00:34:38,0.0,2.60,1.0,N,68,142,1,10.00,3.0,0.5,1.00,0.00,0.3,14.80,2.5,NaN
4,2,2020-04-01 00:13:24,2020-04-01 00:18:26,1.0,1.44,1.0,Y,263,74,1,6.50,0.5,0.5,3.00,0.00,0.3,13.30,2.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238068,2,2020-04-30 23:30:00,2020-04-30 23:39:00,NaN,2.02,NaN,NaN,238,75,0,8.46,0.0,0.5,0.00,0.00,0.3,11.76,NaN,NaN
238069,2,2020-04-30 23:44:00,2020-04-30 23:54:00,NaN,3.30,NaN,NaN,263,230,0,13.46,0.0,0.5,0.00,0.00,0.3,16.76,NaN,NaN
238070,2,2020-04-30 23:25:00,2020-04-30 23:38:00,NaN,6.40,NaN,NaN,137,42,0,22.05,0.0,0.5,0.00,0.00,0.3,25.35,NaN,NaN
238071,2,2020-04-30 23:50:26,2020-05-01 00:28:21,NaN,25.70,NaN,NaN,29,69,0,68.16,0.0,0.5,0.00,6.12,0.3,77.58,NaN,NaN


In [8]:
train_x['tpep_pickup_datetime'] = pd.to_datetime(train_x['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
train_x['tpep_dropoff_datetime'] = pd.to_datetime(train_x['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
train_x['trip_duration'] = (train_x['tpep_dropoff_datetime'] - train_x['tpep_pickup_datetime']).dt.seconds
train_x['PULocationID'].fillna(-1, inplace=True)
train_x['DOLocationID'].fillna(-1, inplace=True)
train_x['tpep_pickup_month'] = train_x['tpep_pickup_datetime'].dt.month
train_x['tpep_dropoff_month'] = train_x['tpep_dropoff_datetime'].dt.month
train_x['tpep_pickup_day_numeric'] = train_x['tpep_pickup_datetime'].dt.day
train_x['tpep_dropoff_day_numeric'] = train_x['tpep_dropoff_datetime'].dt.day
train_x['tpep_pickup_day_name'] = train_x['tpep_pickup_datetime'].dt.day_name()
train_x['tpep_dropoff_day_name'] = train_x['tpep_dropoff_datetime'].dt.day_name()
train_x['tpep_pickup_hour'] = train_x['tpep_pickup_datetime'].dt.hour
train_x['tpep_dropoff_hour'] = train_x['tpep_dropoff_datetime'].dt.hour
train_x['tpep_pickup_day'] = train_x['tpep_pickup_datetime'].dt.strftime("%w").astype(int)
train_x['tpep_dropoff_day'] = train_x['tpep_dropoff_datetime'].dt.strftime("%w").astype(int)
train_y =  (train_x['tpep_dropoff_datetime'] - train_x['tpep_pickup_datetime']).dt.seconds
train_x['tpep_pickup_datetime'] = train_x['tpep_pickup_datetime'].apply(lambda x: int(x.strftime('%Y%m%d')))
train_x['tpep_dropoff_datetime'] = train_x['tpep_dropoff_datetime'].apply(lambda x: int(x.strftime('%Y%m%d')))
train_x = train_x.fillna(0)
train_x = train_x.drop(["tpep_dropoff_day_name"], axis=1)
train_x = train_x.drop(["tpep_pickup_day_name"], axis=1)
train_x = train_x.drop(["store_and_fwd_flag"], axis=1)

print(train_x)

        VendorID  tpep_pickup_datetime  tpep_dropoff_datetime  \
0              1              20200401               20200401   
1              1              20200401               20200401   
2              1              20200401               20200401   
3              1              20200401               20200401   
4              2              20200401               20200401   
...          ...                   ...                    ...   
238068         2              20200430               20200430   
238069         2              20200430               20200430   
238070         2              20200430               20200430   
238071         2              20200430               20200501   
238072         2              20200430               20200430   

        passenger_count  trip_distance  RatecodeID  PULocationID  \
0                   1.0           1.20         1.0            41   
1                   1.0           3.40         1.0            95   
2              

In [9]:
from sklearn.preprocessing import MinMaxScaler
columns = ['PULocationID', 'DOLocationID', 'tpep_pickup_month', 'tpep_pickup_day', 'tpep_pickup_hour', 'trip_duration']
# define min max scaler
scaler = MinMaxScaler()
train_scaled = train_x[columns] = scaler.fit_transform(train_x[columns])

y_train_labels = train_x["trip_duration"].values
x_train_samples = train_x.drop(["trip_duration"], axis=1)
print(x_train_samples)
print("---")
print(y_train_labels)



        VendorID  tpep_pickup_datetime  tpep_dropoff_datetime  \
0              1              20200401               20200401   
1              1              20200401               20200401   
2              1              20200401               20200401   
3              1              20200401               20200401   
4              2              20200401               20200401   
...          ...                   ...                    ...   
238068         2              20200430               20200430   
238069         2              20200430               20200430   
238070         2              20200430               20200430   
238071         2              20200430               20200501   
238072         2              20200430               20200430   

        passenger_count  trip_distance  RatecodeID  PULocationID  \
0                   1.0           1.20         1.0      0.151515   
1                   1.0           3.40         1.0      0.356061   
2              

In [10]:
x_train_samples = np.array(x_train_samples)
y_train_labels = np.array(y_train_labels)
print(x_train_samples)
print(y_train_labels)

[[1.00000000e+00 2.02004010e+07 2.02004010e+07 ... 1.00000000e+00
  5.00000000e-01 3.00000000e+00]
 [1.00000000e+00 2.02004010e+07 2.02004010e+07 ... 1.00000000e+00
  5.00000000e-01 3.00000000e+00]
 [1.00000000e+00 2.02004010e+07 2.02004010e+07 ... 0.00000000e+00
  5.00000000e-01 3.00000000e+00]
 ...
 [2.00000000e+00 2.02004300e+07 2.02004300e+07 ... 2.30000000e+01
  6.66666667e-01 4.00000000e+00]
 [2.00000000e+00 2.02004300e+07 2.02005010e+07 ... 0.00000000e+00
  6.66666667e-01 5.00000000e+00]
 [2.00000000e+00 2.02004300e+07 2.02004300e+07 ... 2.30000000e+01
  6.66666667e-01 4.00000000e+00]]
[0.01424785 0.00931724 0.0062385  ... 0.00902788 0.02633132 0.00996539]


In [ ]:
model = Sequential([
    Dense(units=32, input_shape=(26,1), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=1)
])
model.summary()

model.compile(optimizer=SGD(learning_rate=0.001), loss='mean_squared_error',metrics=['accuracy'])
model.fit(x=x_train_samples, y=y_train_labels, validation_split=0.1, batch_size=1000, epochs=30, shuffle=True, verbose=2)




Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 26, 32)            64        
                                                                 
 dense_12 (Dense)            (None, 26, 32)            1056      
                                                                 
 dense_13 (Dense)            (None, 26, 32)            1056      
                                                                 
 dense_14 (Dense)            (None, 26, 1)             33        
                                                                 
Total params: 2,209
Trainable params: 2,209
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
215/215 - 45s - loss: nan - accuracy: 9.8476e-04 - val_loss: nan - val_accuracy: 0.0030 - 45s/epoch - 208ms/step
Epoch 2/30
215/215 - 48s - loss: nan - accuracy: 9.9876e-04 